In [ ]:
library(dplyr)
library(Seurat)
library(patchwork)
library(here)
library(SeuratObject)
library(Matrix)
library(RColorBrewer)
library(tidyverse)

In [ ]:
#load data from medicago roots inoculated with AMF:

data.dir1 <-"/global/cfs/projectdirs/m342/plant-microbe_shared/Chromium_outs/Chromium_medicago_outs/MBBC001A_ensembl/outs/filtered_feature_bc_matrix" #med28_1
data.dir2 <-"/global/cfs/projectdirs/m342/plant-microbe_shared/Chromium_outs/Chromium_medicago_outs/output_Mt_with_AMF_MBBC022B/outs/filtered_feature_bc_matrix" #med28_2
data.dir4 <-"/global/cfs/projectdirs/m342/plant-microbe_shared/Chromium_outs/Chromium_medicago_outs/MBBC003A_ensembl/filtered_feature_bc_matrix" #med38


med28_1<-Read10X(
  data.dir1,
  gene.column = 2,
  cell.column = 1,
  unique.features = TRUE,
  strip.suffix = FALSE
)

med28_2<-Read10X(
  data.dir2,
  gene.column = 2,
  cell.column = 1,
  unique.features = TRUE,
  strip.suffix = FALSE
)


med38<-Read10X(
  data.dir4,
  gene.column = 2,
  cell.column = 1,
  unique.features = TRUE,
  strip.suffix = FALSE
)

med28_1<- CreateSeuratObject(counts = med28_1, project="med28_1")
VlnPlot(med28_1, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)
med28_1 <- subset(med28_1,nFeature_RNA > 300)
VlnPlot(med28_1, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)

med28_2 <- CreateSeuratObject(counts = med28_2, project="med28_2")
VlnPlot(med28_2, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)
med28_2 <- subset(med28_2, nFeature_RNA > 200)
VlnPlot(med28_2, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)

med38 <- CreateSeuratObject(counts = med38, project="med38")
VlnPlot(med38, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)
med38 <- subset(med38, nFeature_RNA > 300)
VlnPlot(med38, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2, pt.size = .05)

In [ ]:
med28_1 <- SCTransform(med28_1, verbose = TRUE)
med28_1 <- RunPCA(med28_1, npcs = 30, verbose = FALSE)
med28_1 <- RunUMAP(med28_1, reduction = "pca", dims = 1:30)
med28_1 <- FindNeighbors(med28_1, reduction = "pca", dims = 1:30)
med28_1 <- FindClusters(med28_1, resolution = 0.5)

med28_2 <- SCTransform(med28_2, verbose = TRUE)
med28_2 <- RunPCA(med28_2, npcs = 30, verbose = FALSE)
med28_2 <- RunUMAP(med28_2, reduction = "pca", dims = 1:30)
med28_2 <- FindNeighbors(med28_2, reduction = "pca", dims = 1:30)
med28_2 <- FindClusters(med28_2, resolution = 0.5)

med38 <- SCTransform(med38, verbose = TRUE)
med38 <- RunPCA(med38, npcs = 30, verbose = FALSE)
med38 <- RunUMAP(med38, reduction = "pca", dims = 1:30)
med38  <- FindNeighbors(med38, reduction = "pca", dims = 1:30)
med38  <- FindClusters(med38, resolution = 0.5)

In [ ]:
#integrate three datasets from inoculated samples:

med.list<-c(med28_1, med28_2, med38)

med.features <- SelectIntegrationFeatures(object.list = med.list, nfeatures=3000)

all.genes <- lapply(med.list, row.names) %>% Reduce(c, .) %>% unique()

med.list <-PrepSCTIntegration(med.list,assay = NULL,anchor.features=med.features,sct.clip.range = NULL,verbose = TRUE)

med.anchors <- FindIntegrationAnchors(object.list = med.list, normalization.method = "SCT", anchor.features = med.features)

med <- IntegrateData(anchorset = med.anchors, normalization.method = "SCT", verbose = TRUE, features.to.integrate = all.genes)

In [ ]:
DefaultAssay(med)<-'integrated'
med <- RunPCA(med, npcs = 30, verbose = FALSE)
med <- RunUMAP(med, reduction = "pca", dims = 1:30, metric='cosine')
med <- FindNeighbors(med, reduction = "pca", dims = 1:30)
med <- FindClusters(med, resolution = 0.5)

In [ ]:
#visualize integrated dataset by cell type and by dataset

#name clusters:
med <- RenameIdents(object = med,
"0" = "Cortex 1",
"1" = "Cortex 2",
"2" = "Cortex 3",
"3" = "Cortex 4",
"4" = "Pericycle",
"5" = "Stele", 
"6" = "Cortex 5", 
"7" = "Endodermis", 
"8" = "LRP", 
"9" = "Cortex 6",
"10" = "Cortex 7", 
"11" = "Unknown", 
"12" = "Colonized Cortex",
"13" = "Cortex 8",
"14" = "Phloem", 
"15" = "Xylem")

colors = c("#FF9999",
"#FF6666",
"#CC99FF",
"#FF6699",
"#ff9933",
"#FFCC66",
"#CC6699",
"#99CCCC",
"#66CC99",
"#FF99FF",
"#FF99CC",
"#99CCFF",
"#990000",
"#FFCCCC",
"#CC9999",
"#996666")

cell_type<-Idents(object = med)

#Figure 1b
DimPlot(med, reduction = "umap", group.by = "cell_type", label = TRUE, pt.size = 1, cols=colors)
DimPlot(med, group.by="orig.ident", shuffle=TRUE)

In [ ]:
#Figure S1
# reorder clusters
med_condensed@active.ident <- factor(med_condensed@active.ident, 
                            levels=c("Unknown","LRP","Endodermis","Pericycle", "Xylem", "Stele", "Phloem", "Colonized Cortex",
                                     "Cortex"))

plot<-tiff("cluster_dotplot.tiff",width=1000,height = 300)
DotPlot(
  med_condensed,
  assay = "RNA",
  features = c("A17-----------MTR-3g107870", #cortex
               "A17-----------MTR-3g093270", #amf
               "A17-----------MTR-4g081190", #amf
               "A17-----------MTR-8g022270", #amf
               "A17-----------MTR-8g068265", #amf
               "A17-----------MTR-6g444980", #phloem
               "A17-----------MTR-8g069955", #stele
               "A17-----------MTR-1g101830", #xylem
               "A17-----------MTR-8g036085", #xylem 
               "A17-----------MTR-1g041695", #pericycle/central cylinder
               "A17-----------MTR-1g075640", #pericycle/central cylinder
               "A17-----------MTR-7g074650", #endodermis scarecrow
               "A17-----------MTR-2g098180", #meristematic (LRP)
               "A17-----------MTR-7g080460"  #meristematic (LRP)
), 
  cols = c("lightgrey", "red"),
  dot.scale = 8,
  scale = TRUE,
  scale.max = 30
)
dev.off()

In [ ]:
#to subset colonized cells based on literature marker genes:

AMF_genes <- c("A17-----------MTR-7g045370",
               "A17-----------MTR-3g109610",
               "A17-----------MTR-1g028600",
               "A17-----------MTR-8g022270",
               "A17-----------MTR-4g081190",
               "A17-----------MTR-3g107870",
               "A17-----------MTR-3g093270",
               "A17-----------MTR-7g062580",
               "A17-----------MTR-8g006790",
               "A17-----------MTR-7g092620",
               "A17-----------MTR-3g058840",
               "A17-----------MTR-3g115940",
               "A17-----------MTR-3g093430",
               "A17-----------MTR-1g050525",
               "A17-----------MTR-8g068265",
               "A17-----------MTR-7g027190")

DefaultAssay(med)<-'RNA'
med <- AddModuleScore(med, features = list(AMF_genes), name='colonized_cells')

#to visualize module scores:
FeaturePlot(med, features = "colonized_cells1", pt.size = 2, label = FALSE, repel = TRUE) + scale_colour_gradientn(colours = c("red","red","yellow","green","lightblue"), values = c(1.0,0.8,0.6,0.4,0.2,0.175, 0.15,0.1))

#subset cells >95th percentile to new object:
#colonized_cluster_cells<-subset(rownames for which 'colonized_cells1' is in the 95th percentile)
colonized_cells <- subset(x = med, subset = colonized_cells1 > quantile(med$colonized_cells1,0.95))

DimPlot(colonized_cells, pt.size=3, label.size= 10, label = FALSE)

#recluster colonized subset
DefaultAssay(colonized_cells)<-'integrated'
colonized_cells <- RunPCA(colonized_cells, npcs = 30, verbose = FALSE)
colonized_cells <- RunUMAP(colonized_cells, reduction = "pca", dims = 1:30, metric='correlation')
colonized_cells <- FindNeighbors(colonized_cells, reduction = "pca", dims = 1:30)
colonized_cells <- FindClusters(colonized_cells, resolution = 0.5) 

#Figure 3b (left)

DimPlot(colonized_cells, pt.size = 3,label.size= 10, label = FALSE)  

In [ ]:
#set colonized cluster as sobj for ggplots:
sobj<-colonized_cells

#ggplot function (for Figure 3b)
get_reduction <- function(sobj, reduction_name) {
  rd <- Embeddings(sobj, reduction = reduction_name) %>% 
    as_tibble(rownames = "Cell")
}

get_cell_data <- function(sobj, features = NULL, melt = TRUE) {
  md <- as_tibble(sobj@meta.data, rownames = "Cell")
  reducs <- names(sobj@reductions)
  rd <- map(reducs, get_reduction, sobj = sobj) %>% reduce(left_join)
  combined_data <- left_join(md, rd)
  if(!is.null(features)) {
    exp_data <- GetAssayData(sobj[features,], slot = "counts", assay = "RNA") %>% 
      Matrix::t() %>% 
      as_tibble(rownames = "Cell")
    if(melt) {
      exp_data <- gather(exp_data, "Locus", "Counts", -Cell)
    }
    combined_data <- left_join(combined_data, exp_data)
  }
  return(combined_data)
}

cell_data <- get_cell_data(sobj)

In [ ]:
#Figure 3b (right)
## Plot by expression

features <- c("A17-----------MTR-3g107870") #ABCG59 early
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_ABCG59_early.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

features <- c("A17-----------MTR-8g068265") #DXS2 mid
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_DXS2_mid.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

features <- c("A17-----------MTR-7g045370") #ccd7 mid
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_CCD7_mid.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-3g109610") #ccd1 mid
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_CCD1_mid.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-8g006790") #ATPase late
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_ATPase_late.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-1g028600") #pt4 late
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("CC_PT4_late.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


In [ ]:
#ggplot function (for Figure 3a, Figure 4b and Figure 4c)

sobj<- med

#ggplot function
get_reduction <- function(sobj, reduction_name) {
  rd <- Embeddings(sobj, reduction = reduction_name) %>% 
    as_tibble(rownames = "Cell")
}

get_cell_data <- function(sobj, features = NULL, melt = TRUE) {
  md <- as_tibble(sobj@meta.data, rownames = "Cell")
  reducs <- names(sobj@reductions)
  rd <- map(reducs, get_reduction, sobj = sobj) %>% reduce(left_join)
  combined_data <- left_join(md, rd)
  if(!is.null(features)) {
    exp_data <- GetAssayData(sobj[features,], slot = "counts", assay = "RNA") %>% 
      Matrix::t() %>% 
      as_tibble(rownames = "Cell")
    if(melt) {
      exp_data <- gather(exp_data, "Locus", "Counts", -Cell)
    }
    combined_data <- left_join(combined_data, exp_data)
  }
  return(combined_data)
}

cell_data <- get_cell_data(sobj)


#Figure 3a:

features <- c("A17-----------MTR-3g107870") #ABCG59 early
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig3_ABCG59_early.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

features <- c("A17-----------MTR-8g068265") #DXS2 mid
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=2) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig3_DXS2_mid.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)



#Figure 4b, Figure 4c:

## Plot by expression
features <- c("A17-----------MTR-3g097560") #PDR1 homolog
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)), size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_PDR1.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

features <- c("A17-----------CCAMK") #CCAMk
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_CCAMK.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-1g028600") #pt4
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_pt4.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-7g068600") #myb1
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_myb1.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

#genes of unknown function highly correlated with known marker genes

features <- c("A17-----------MTR-8g006790") #ATPase
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_ATPase_.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-6g037750") #LRR domain protein
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_lrr.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-3g093430") #ABC19
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_ABC19.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


features <- c("A17-----------MTR-3g058840") #other LRR
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.1) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Fig4_med_LRR2.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)


In [ ]:
#Supp Sheet 4
#DEG anaylsis between colonized cluster 14 and rest of cortical cells
med_condensed <- med
med_condensed <- RenameIdents(object = med_condensed,
"Cortex 1" = "Cortex",
"Cortex 2" = "Cortex",
"Cortex 3" = "Cortex",
"Cortex 4" = "Cortex",
"Cortex 6" = "Cortex",
"Cortex 7" = "Cortex",
"Cortex 8" = "Cortex")

med_condensed <- FindMarkers(med_condensed, ident.1 = "Colonized Cortex", ident.2 = "Cortex", only.pos = TRUE, assay = "RNA",verbose = FALSE)
write.csv(med_condensed, "colonized_markers_June27.csv")